In [ ]:
# Build a Network that takes ux_hat, uy_hat, T_hat as input and gives sigma and tau as outputs


import tensorflow as tf
tf.random.set_seed(10)
import random
random.seed()

from keras import layers

from tensorflow.python.keras.backend import clear_session

from keras.layers import Input, Convolution2D, Convolution1D, MaxPooling2D, Dense, Dropout, \
                          Flatten, concatenate, Activation, Reshape, \
                          UpSampling2D,ZeroPadding2D
from keras.layers import Dense
from keras import Sequential
import h5py
import keras
from scipy.io import loadmat,savemat

In [ ]:
# Load the files from the RB simulation

with h5py.File('input_normalized.h5', 'r') as hf:
    inpit_normalized = np.array(hf['input_normalized'])

with h5py.File('input_normalized_val.h5', 'r') as hf:
    inpit_normalized = np.array(hf['input_normalized_val'])

with h5py.File('output_normalized.h5', 'r') as hf:
    inpit_normalized = np.array(hf['output_normalized'])

with h5py.File('output_normalized_val.h5', 'r') as hf:
    inpit_normalized = np.array(hf['output_normalized_val'])

In [ ]:
print(input_normalized.shape)
print(input_normalized_val.shape)

print(output_normalized.shape)
print(output_normalized_val.shape)

In [ ]:
def build_model(conv_depth, kernel_size, hidden_size, n_hidden_layers, lr):

    model = keras.Sequential([

            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation, input_shape=(Nlon,Nlat,n_channels)),
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            ] + [keras.layers.Dense(hidden_size, activation='sigmoid') for i in range(n_hidden_layers)] +

            [

            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            layers.Convolution2D(5, kernel_size, padding='same', activation=None)
            ]
            )
    optimizer= keras.optimizers.Adam(lr=lr)


    model.compile(loss='mean_squared_error', optimizer = optimizer)

    return model

In [ ]:
# Set model hyperparameters

params = {'conv_depth': 32, 'hidden_size': 5000,
              'kernel_size': 5, 'lr': 0.00005, 'n_hidden_layers': 0}

model = build_model(**params)
print(model.summary())

In [ ]:
model.fit(input_normalized, output_normalized, batch_size=batch_size, shuffle=True, verbose=1, epochs=100,validation_data=(input_normalized_val,output_normalized_val))

In [ ]:
model.save('./CNN')
model.save_weights('./weights_CNN')

In [ ]:
# save model as zip file to download

!zip -r 'CNN.zip' 'CNN'

In [ ]:
pred = model.predict(input_normalized_val[:,:,:,:])
correct = output_normalized_val[:,:,:,:]

In [ ]:
# calculate the correlation coefficient between the prediction and the true values in the validation set

x = pred.flatten()
y = correct.flatten()
print(np.corrcoef(x,y)[0,1])